In [2]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = '../dataset/all.reef.csv'
benthic_data = pd.read_csv(file_path)

# Display basic information and check for missing values
print(benthic_data.info())
print(benthic_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   P_CODE       806 non-null    object 
 1   NRM_REGION   806 non-null    object 
 2   REEF         806 non-null    object 
 3   DEPTH        806 non-null    int64  
 4   VISIT_NO     806 non-null    int64  
 5   Date         806 non-null    object 
 6   subregion    806 non-null    object 
 7   A            806 non-null    float64
 8   AB           806 non-null    float64
 9   HC           806 non-null    float64
 10  SC           806 non-null    float64
 11  CoralCover   806 non-null    float64
 12  MA           806 non-null    float64
 13  Acr          806 non-null    float64
 14  DISTURBANCE  739 non-null    object 
 15  juv5.d.nf    697 non-null    float64
dtypes: float64(8), int64(2), object(6)
memory usage: 100.9+ KB
None
  P_CODE NRM_REGION      REEF  DEPTH  VISIT_NO        Date subregion  

In [3]:
# Rename columns for clarity
benthic_data.rename(columns={
    'P_CODE': 'program_code',
    'NRM_REGION': 'region',
    'REEF': 'reef_location',
    'DEPTH': 'depth_in_meter',
    'VISIT_NO': 'visit_number',
    'Date': 'observation_date',
    'subregion': 'subregion',
    'A': 'algae_cover',
    'AB': 'sediment_cover',
    'HC': 'hard_coral_cover',
    'SC': 'soft_coral_cover',
    'CoralCover': 'sum_of_hard_soft_cover',
    'MA': 'macroalgae_cover',
    'Acr': 'acropora_cover',
    'DISTURBANCE': 'disturbance_type',
    'juv5.d.nf': 'juvenile_coral_density'
}, inplace=True)

# Display the first few rows of the dataset with new column names
print(benthic_data.head())

  program_code    region reef_location  depth_in_meter  visit_number  \
0           IN  Burdekin      Havannah               2             1   
1           IN  Burdekin      Havannah               2             2   
2           IN  Burdekin      Havannah               2             3   
3           IN  Burdekin      Havannah               2             5   
4           IN  Burdekin      Havannah               2             7   

  observation_date subregion  algae_cover  sediment_cover  hard_coral_cover  \
0       23/08/2005  Burdekin    76.187500        7.187500         12.562500   
1       24/08/2006  Burdekin    75.268365        5.378159         15.665533   
2        3/07/2007  Burdekin    70.348906        7.821541         16.949005   
3       19/06/2009  Burdekin    68.000000        8.000000         19.187500   
4       17/08/2011  Burdekin    63.875000       15.812500         15.187500   

   soft_coral_cover  sum_of_hard_soft_cover  macroalgae_cover  acropora_cover  \
0          

In [4]:
# Replace missing values in the disturbance_type column with 'unk'
benthic_data['disturbance_type'] = benthic_data['disturbance_type'].fillna('unk')

# Impute missing values in the juvenile_coral_density column with the mean
benthic_data['juvenile_coral_density'] = benthic_data['juvenile_coral_density'].fillna(benthic_data['juvenile_coral_density'].mean())

In [5]:
# Calculate and create the Coral Health Index (CHI)
benthic_data['coral_health_index'] = (benthic_data['hard_coral_cover'] + benthic_data['soft_coral_cover']) / (benthic_data['hard_coral_cover'] + benthic_data['soft_coral_cover'] + benthic_data['macroalgae_cover'])

In [6]:
# Remove rows where disturbance_type is "b" because we not sure what "b" is stand for
benthic_data = benthic_data[benthic_data['disturbance_type'] != 'b']

In [7]:
# Encode Disturbance Impact Score (DIS)
disturbance_mapping = {'n': 0, 'f': 1, 'd': 2, 's': 3, 'c': 4, 'm': 5, 'unk': 6}

# Ensure that the dataset is not a view
benthic_data = benthic_data.copy()

# Apply the mapping using .loc to avoid the SettingWithCopyWarning
benthic_data.loc[:, 'disturbance_impact_score'] = benthic_data['disturbance_type'].map(disturbance_mapping)

In [8]:
# Convert the observation_date column to datetime format
benthic_data['observation_date'] = pd.to_datetime(benthic_data['observation_date'], format='%d/%m/%Y')

# Extract season from the observation_date column
benthic_data['season'] = benthic_data['observation_date'].dt.month % 12 // 3 + 1

In [9]:
print(benthic_data.head())


  program_code    region reef_location  depth_in_meter  visit_number  \
0           IN  Burdekin      Havannah               2             1   
1           IN  Burdekin      Havannah               2             2   
2           IN  Burdekin      Havannah               2             3   
3           IN  Burdekin      Havannah               2             5   
4           IN  Burdekin      Havannah               2             7   

  observation_date subregion  algae_cover  sediment_cover  hard_coral_cover  \
0       2005-08-23  Burdekin    76.187500        7.187500         12.562500   
1       2006-08-24  Burdekin    75.268365        5.378159         15.665533   
2       2007-07-03  Burdekin    70.348906        7.821541         16.949005   
3       2009-06-19  Burdekin    68.000000        8.000000         19.187500   
4       2011-08-17  Burdekin    63.875000       15.812500         15.187500   

   soft_coral_cover  sum_of_hard_soft_cover  macroalgae_cover  acropora_cover  \
0          